## Frequent Item Sets 

### Efficiency considerations

1. I read the entire data set into main memory to speed up the implementation. However, if the data set is huge, we could read one line at a time from disk, process to count frequency then go back and read next line.

2. The time taken upto identification of frequent quadruples is within 3 mins on local machine. For five and over, it takes longer. The biggest time consumption here is to look through all candidate quads for each five group and check if they are a frequent quad. If we had a cluster and memory is not a constraint, this check could be avoided to speed up the algorithm.


In [1]:
#Read the dataset in to main memory and store in raw_data
import pandas as pd
import os
#import nltk
#nltk.download('punkt')

support_level = 4

with open('C:/Users/akolachi/Globality Retail25K/retail_25k.dat',mode ='r') as fp:
    line = fp.readline().strip()
    raw_data = {}
    counter = 0
    while (line):
        raw_data[counter] = line.split()
        counter = counter+1
        line = fp.readline().strip()

In [2]:
len(raw_data)

25000

25000 transactions in the database

In [3]:
#create a dataframe to capture frequency of each item in different itemsets.
# In the current block we will set the keys of the dataframe to individual items and leave
# values as null lists. The values will be updated once frequent item sets are computed
individual_items_count = {}
for k, v in raw_data.items():
    for i in range(len(v)):
        if v[i] in individual_items_count.keys():
           continue
        else:
            individual_items_count[v[i]] = []
    else:
        continue


In [4]:
##Function to prune count of item sets to frequent item sets based on support level.
## x -  dict of item counts 
## support_level = minimum no of transactions the itemset needs to be in to be considered frequent
## y - returns dict of frequent items
def prune(x):
    y = {}    
    for k,v in x.items():
        if v >= support_level:
            y[k] = v
        else: 
            continue
    return y

In [5]:
## Add number of itemsets the item is in to individual_item_count
# x - dict of frequent item set 
# individual_items_count - dict of individual_items_count initiated earlier with keys as individual items
#no_items_in_tuple - size of the itemset (1,2,3,4, ...)

def capture_item_count(x,individual_items_count, no_of_items_in_tuple):
       
    #initialize to 0 a new entry into the list of values for the itemset
    for k, v in individual_items_count.items():
        individual_items_count[k].insert(no_of_items_in_tuple, 0)
    
    
    #if the itemset is size one, number of itemsets the item is in = number of transactions item is in
    if no_of_items_in_tuple == 1:
        for k,v in x.items():
            individual_items_count[k][no_of_items_in_tuple-1] = v
    
    #for bigger itemset sizes, scan through the frequent itemset dict and increment count everytime the item is found 
    #K is a tuple of length = size of itemset
    else:
        for k,v in x.items():
            for i in range(no_of_items_in_tuple):
                individual_items_count[k[i]][no_of_items_in_tuple-1] +=1
                

    return individual_items_count
    

In [6]:
#check that each item is present only once in each transaction
# This didnot print anything to screen. So, confirmed no duplicates within each transaction
for k, v in raw_data.items():
    cnt = {}
    v = list(map(int, v))
    v.sort()
    v = list(map(str, v))
    for i in range(len(v)):
        if v[i] in cnt.keys():
            cnt[v[i]] +=1
        else:
            cnt[v[i]] =1
    for k,v in cnt.items():
        if v >1:
            print(k)

In [7]:
#Count the number of transactions each item is in to single_count
single_count = {}

for k, v in raw_data.items():
    #make sure that items are sorted, so that when pairs and triples are created, the combination is not repeated
    #for ex., (1,2) and (2,1) is one pair only. (2,1) can be avoided by sorting
    v = list(map(int, v))
    v.sort()
    v = list(map(str,v))
    #use v only if v has more elements than the itemset size we are trying to get. (in this case, 1)
    if len(v) >=1: 
        for i in range(len(v)):
            if v[i] in single_count.keys():
               single_count[v[i]] +=1
            else:
                single_count[v[i]] = 1
        else:
            continue

# prune the counts to get frequent items           
single_freq_items = prune(single_count)
#update the first element of individual_items_count with each frequent items frequency. If item is not frequent, it will be set to 0
individual_items_count = capture_item_count(single_freq_items,individual_items_count,no_of_items_in_tuple =1 )

In [8]:
len(single_count)

11427

In [9]:
len(single_freq_items)

7232

In [10]:
#Count number of candidate pairs of items into double_count dict
double_count = {}
for k, v in raw_data.items():
    v = list(map(int, v))
    v.sort()
    v = list(map(str,v))
    #use v only if it has more than 2 items required for itemset size 2
    if len(v) >=2:
        #only use item if it is a singleton frequent item
        for i in range(len(v)-1):
            if individual_items_count[v[i]][0]>0:
                for j in range(i+1,len(v)):
                    if individual_items_count[v[j]][0]>0:
                        #create a candidate pair for all combinations of items in v that are singleton frequent
                        if (v[i],v[j]) in double_count.keys():
                            double_count[(v[i],v[j])] +=1
                        else:
                            double_count[(v[i],v[j])] = 1
                    else:
                        continue
            else:
                continue

#prune the counts to get frequent pairs
double_freq_items = prune(double_count)  

#update the second element of individual_items_count with no of frequent pairs each item is in. If item is not in any pair, it will be set to 0
individual_items_count = capture_item_count(double_freq_items,individual_items_count,no_of_items_in_tuple =2 )

In [11]:
len(double_freq_items)

54715

In [12]:
#count number of candidate triples and capture in triple_count dict
triple_count = {}
for k, v in raw_data.items():

    v = list(map(int, v))
    v.sort()
    v = list(map(str, v))
    candidate_triple_single = []
    
    #process only if there are atleast 3 items in transaction
    if len(v) >=3:
        for i in range(len(v)):
            #each item should be in atleast two frequent item pairs to be a frequent triple
            #capture all such items from transaction into candidate_triple_single list
                try:
                    if individual_items_count[v[i]][1] >=2:
                        candidate_triple_single.append(v[i])
                        
                except KeyError:
                    continue
        
        # only if there are atleast 3 items in the list, there is a possibility of making a candidate triple  
        if len(candidate_triple_single) >=3:
            #make all possible triples from the transaction
            for p in range(len(candidate_triple_single)-2):
                for q in range(p+1,len(candidate_triple_single)-1):
                    for r in range(q+1, len(candidate_triple_single)):
                        a = (candidate_triple_single[p],candidate_triple_single[q],candidate_triple_single[r])
                        candidate_triple_double = []
                        #for each triple, check if ALL underlying pairs are frequent pairs.
                        for i in range(len(a)-1):
                            for j in range(i+1,len(a)):
                                if (a[i],a[j]) in double_freq_items.keys():
                                    candidate_triple_double.append((a[i],a[j]))
                                else:
                                    continue
                        
                        #If the triple makes three frequent pairs, it can be considered a candidate triple
                        if len(candidate_triple_double)== 3:
                            if a in triple_count.keys():
                                triple_count[a] += 1
                            else:
                                triple_count[a] =1
                        else: 
                            continue

#prune the counts to get frequent pairs                           
triple_freq_items = prune(triple_count)

#update the third element of individual_items_count with no of frequent triples each item is in. If item is not in any triple, it will be set to 0
individual_items_count = capture_item_count(triple_freq_items,individual_items_count,no_of_items_in_tuple =3)

In [13]:
len(triple_freq_items)

76151

In [14]:
#Similar to frequent triple, we now:
#check for each item being in atleast 3 frequent triples
#create all possible quads
#each quad can make four triples. All four triples have to be frequent for the quad to be a candidate itemset

quad_count = {}
for k, v in raw_data.items():

    v = list(map(int, v))
    v.sort()
    v = list(map(str, v))
    candidate_quad_single = []
    
    
    if len(v) >=4:
        for i in range(len(v)):
                try:
                    if individual_items_count[v[i]][2] >=3:
                        candidate_quad_single.append(v[i])
                        
                except KeyError:
                    continue
            
        if len(candidate_quad_single) >=4:
            for p in range(len(candidate_quad_single)-3):
                for q in range(p+1,len(candidate_quad_single)-2):
                    for r in range(q+1, len(candidate_quad_single)-1):
                        for l in range(r+1,len(candidate_quad_single)):
                            a = (candidate_quad_single[p],candidate_quad_single[q],candidate_quad_single[r],candidate_quad_single[l])
                            candidate_quad_triple = []
                            for i in range(len(a)-2):
                                for j in range(i+1,len(a)-1):
                                    for k in range(j+1, len(a)):
                                        if (a[i],a[j],a[k]) in triple_freq_items.keys():
                                            candidate_quad_triple.append((a[i],a[j],a[k]))
                                        else:
                                            continue

                            if len(candidate_quad_triple)== 4:
                                if a in quad_count.keys():
                                    quad_count[a] += 1
                                else:
                                    quad_count[a] =1
                            else: 
                                continue

quad_freq_items = prune(quad_count)
individual_items_count = capture_item_count(quad_freq_items,individual_items_count,no_of_items_in_tuple =4)

In [15]:
len(quad_freq_items)

56225

In [16]:
#Similar to triple and quad
five_count = {}
for k, v in raw_data.items():

    v = list(map(int, v))
    v.sort()
    v = list(map(str, v))
    candidate_five_single = []
    
    
    if len(v) >=5:
        for i in range(len(v)):
            if individual_items_count[v[i]][3] >=4:
                candidate_five_single.append(v[i])
                        
                
        if len(candidate_five_single) >=5:
            for p in range(len(candidate_five_single)-4):
                for q in range(p+1,len(candidate_five_single)-3):
                    for r in range(q+1, len(candidate_five_single)-2):
                        for l in range(r+1,len(candidate_five_single)-1):
                            for m in range(r+1,len(candidate_five_single)):
                                a = (candidate_five_single[p],candidate_five_single[q],candidate_five_single[r],candidate_five_single[l],candidate_five_single[m] )
                                candidate_five_quad = []
                                for i in range(len(a)-3):
                                    for j in range(i+1,len(a)-2):
                                        for k in range(j+1, len(a)-1):
                                            for h in range(k+1, len(a)):
                                                if (a[i],a[j],a[k],a[h]) in quad_freq_items.keys():
                                                    candidate_five_quad.append((a[i],a[j],a[k],a[h]))
                                                else:
                                                    continue

                                if len(candidate_five_quad)== 5:
                                    if a in five_count.keys():
                                        five_count[a] += 1
                                    else:
                                        five_count[a] =1
                                else: 
                                    continue

five_freq_items = prune(five_count)
individual_items_count = capture_item_count(five_freq_items,individual_items_count,no_of_items_in_tuple =5)

In [17]:
len(five_freq_items)

34608

In [18]:
six_count = {}
for k, v in raw_data.items():

    v = list(map(int, v))
    v.sort()
    v = list(map(str, v))
    candidate_six_single = []
    
    
    if len(v) >=6:
        for i in range(len(v)):
            if individual_items_count[v[i]][4] >=5:
                candidate_six_single.append(v[i])
                        
                
        if len(candidate_six_single) >=6:
            for p in range(len(candidate_six_single)-5):
                for q in range(p+1,len(candidate_six_single)-4):
                    for r in range(q+1, len(candidate_six_single)-3):
                        for l in range(r+1,len(candidate_six_single)-2):
                            for m in range(r+1,len(candidate_six_single)-1):
                                for m1 in range(m,len(candidate_six_single)):
                                    a = (candidate_six_single[p],candidate_six_single[q],candidate_six_single[r],candidate_six_single[l],candidate_six_single[m],candidate_six_single[m1]  )
                                    candidate_six_five = []
                                    for i in range(len(a)-4):
                                        for j in range(i+1,len(a)-3):
                                            for k in range(j+1, len(a)-2):
                                                for h in range(k+1, len(a)-1):
                                                    for h1 in range(h+1, len(a)):
                                                        if (a[i],a[j],a[k],a[h],a[h1]) in five_freq_items.keys():
                                                            candidate_six_five.append((a[i],a[j],a[k],a[h],a[h1]))
                                                        else:
                                                            continue

                                    if len(candidate_six_five)== 5:
                                        if a in six_count.keys():
                                            six_count[a] += 1
                                        else:
                                            six_count[a] =1
                                    else: 
                                        continue


six_freq_items = prune(six_count)
individual_items_count = capture_item_count(six_freq_items,individual_items_count,no_of_items_in_tuple =6)

In [19]:
len(six_count)

473

In [20]:
# Checking the results for one item set. We retrieve transaction ids which have one itemset from above
# No of transactions should match the itemset count. I verified it for a few combinations. 
#So, the result makes sense
cnt = []
for k,v in  raw_data.items():
    if ('36' in v and '37' in v and '38' in v and  '39' in v and  '41' in v  and '42' in v): 
        cnt.append(k)

In [21]:
cnt

[3, 8148, 11192]

In [22]:
raw_data[cnt[2]]

['36',
 '37',
 '38',
 '39',
 '41',
 '42',
 '44',
 '45',
 '249',
 '267',
 '408',
 '766',
 '2554',
 '2812',
 '3660',
 '3775',
 '5767',
 '8718']

In [23]:
len(six_freq_items)

0

In [24]:
#Since there are no frequent itemsets of size 6, there cant be any frequent itemsets of size 7 or above.
#However, just to confirm-
seven_count = {}
for k, v in raw_data.items():

    v = list(map(int, v))
    v.sort()
    v = list(map(str, v))
    candidate_seven_single = []
    
    
    if len(v) >=7:
        for i in range(len(v)):
            if individual_items_count[v[i]][5] >=6:
                candidate_seven_single.append(v[i])
                        
                
        if len(candidate_seven_single) >=7:
            for p in range(len(candidate_seven_single)-6):
                for q in range(p+1,len(candidate_seven_single)-5):
                    for r in range(q+1, len(candidate_seven_single)-4):
                        for l in range(r+1,len(candidate_seven_single)-3):
                            for m in range(r+1,len(candidate_seven_single)-2):
                                for m1 in range(m,len(candidate_seven_single)-1):
                                    for m2 in range(m1,len(candidate_seven_single)):
                                        a = (candidate_seven_single[p],candidate_seven_single[q],candidate_seven_single[r],candidate_seven_single[l],candidate_seven_single[m],candidate_seven_single[m1],candidate_seven_single[m2])
                                        candidate_seven_six = []
                                        for i in range(len(a)-5):
                                            for j in range(i+1,len(a)-4):
                                                for k in range(j+1, len(a)-3):
                                                    for h in range(k+1, len(a)-2):
                                                        for h1 in range(h+1, len(a)-1):
                                                            for h2 in range(h1+1, len(a)):
                                                                if (a[i],a[j],a[k],a[h],a[h1], a[h2]) in six_freq_items.keys():
                                                                    candidate_seven_six.append((a[i],a[j],a[k],a[h],a[h1],a[h2]))
                                                                else:
                                                                    continue

                                        if len(candidate_seven_six)== 6:
                                            if a in seven_count.keys():
                                                seven_count[a] += 1
                                            else:
                                                seven_count[a] =1
                                        else: 
                                            continue


seven_freq_items = prune(seven_count)
individual_items_count = capture_item_count(seven_freq_items,individual_items_count,no_of_items_in_tuple =7)

In [25]:
len(seven_freq_items)

0

In [26]:
#function to convert frequent itemsets into dataframe
def convert_to_df(x, size):
    y = pd.DataFrame.from_dict(x,orient = "index", columns = ["Frequency"])
    y["Itemset_size"] = size
    y["Itemset"] = y.index
    y = y[["Itemset_size","Frequency","Itemset"]]
    return y

In [27]:
#append dataframes to one dataframe
#single = convert_to_df(single_freq_items,1)
#double = convert_to_df(double_freq_items,2)
triple = convert_to_df(triple_freq_items,3)
quad = convert_to_df(quad_freq_items,4)
five = convert_to_df(five_freq_items,5)
final_result = pd.concat([triple,quad,five], ignore_index= True)
final_result.to_csv("C:/Users/akolachi/Globality Retail25K/frequent_itemsets_final_output.csv")

In [28]:
final_result.head()

,Itemset_size,Frequency,Itemset
0,3,5,"(30, 31, 32)"
1,3,17,"(36, 37, 38)"
2,3,10,"(36, 37, 39)"
3,3,8,"(36, 37, 41)"
4,3,5,"(36, 37, 42)"
